# SQL Assignment

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")

## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [127]:
%%time

query = """ 
        SELECT p.Name AS Director_Name, 
               m.title AS Movie_Name, 
               m.year AS Release_Year
        FROM Movie m
        
        
        JOIN M_Director m_d ON m_d.MID = m.MID
        JOIN Person p ON p.PID = m_d.PID
        JOIN M_Genre m_g ON m_g.MID = m.MID
        JOIN Genre g ON g.GID = m_g.GID
        
        
        WHERE 
        g.Name LIKE '%Comedy%' 
        AND
        m.year%4=0 
        AND 
        m.year%100!=0 
        AND 
        m.year%4=0 
        OR 
        m.year%100=0 
        AND 
        m.year%400=0
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
q2.head()

(393, 3)
CPU times: user 98.3 ms, sys: 2.98 ms, total: 101 ms
Wall time: 103 ms


,Director_Name,Movie_Name,Release_Year
0,Roland Emmerich,2012,I 2009
1,Milap Zaveri,Mastizaade,2016
2,Danny Leiner,Harold & Kumar Go to White Castle,2004
3,Anurag Kashyap,Gangs of Wasseypur,2012
4,Frank Coraci,Around the World in 80 Days,2004


## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [40]:
%%time

query = """
        SELECT p.Name as Actor_Name from Person p
        
        JOIN M_Cast m_c ON TRIM(m_c.PID) = TRIM(p.PID)
        JOIN Movie m ON m.MID = m_c.MID
        
        WHERE m.title = 'Anand'
        
        """

q2 = pd.read_sql_query(query, conn)    
print(q2.shape)
q2.head()

(17, 1)
CPU times: user 297 ms, sys: 11.4 ms, total: 308 ms
Wall time: 309 ms


,Actor_Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo


## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [42]:
%%time

query = """
        SELECT p.Name as Actor_Name from Person p
        
        JOIN M_Cast m_c ON TRIM(m_c.PID) = TRIM(p.PID)
        JOIN Movie m ON m.MID = m_c.MID
        
        WHERE m.year < 1970 OR m.year > 1990
        
        """

q3 = pd.read_sql_query(query, conn)    
print(q3.shape)
q3.head()

(67747, 1)
CPU times: user 10min 7s, sys: 1.66 s, total: 10min 8s
Wall time: 10min 14s


,Actor_Name
0,Christian Bale
1,Cate Blanchett
2,Benedict Cumberbatch
3,Naomie Harris
4,Andy Serkis


## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [49]:
%%time

query = """
        
        SELECT p.Name AS Director_Name, 
               COUNT(*) AS No_of_Movies
        FROM Person p
        
        JOIN M_Director m_d ON m_d.PID = p.PID
        
        GROUP BY TRIM(m_d.PID) HAVING COUNT(*) >= 10 ORDER BY No_of_Movies DESC
        
        """

q4 = pd.read_sql_query(query, conn)    
print(q4.shape)
q4.head()

(58, 2)
CPU times: user 34.6 ms, sys: 2.18 ms, total: 36.8 ms
Wall time: 35.2 ms


,Director_Name,No_of_Movies
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Ram Gopal Varma,30
3,Priyadarshan,30
4,Vikram Bhatt,29


## Q5.a --- For each year, count the number of movies in that year that had only female actors.

In [110]:
%%time

query = """
        SELECT m.Year AS Release_Year, 
               COUNT(*) AS No_of_Movies
        FROM Movie m
        
        JOIN( 
                SELECT DISTINCT MID FROM M_Cast 
                WHERE MID NOT IN
                (
                    SELECT m_c.MID FROM M_Cast m_c
                    JOIN Person p ON p.PID = TRIM(m_c.PID)
                    WHERE p.Gender = 'Male'
                )
            )
        c on c.MID = m.MID
        GROUP BY m.Year
        
        """

q5a = pd.read_sql_query(query, conn)    
print(q5a.shape)
q5a.head()

(7, 2)
CPU times: user 191 ms, sys: 6.57 ms, total: 198 ms
Wall time: 199 ms


,Release_Year,No_of_Movies
0,1939,1
1,1999,1
2,2000,1
3,2009,1
4,2012,1


## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [113]:
%%time

query = """
        SELECT m.Year AS Release_Year, 
            COUNT(m.mid) AS No_of_Movies, 
            f.Female_Only_Movies,
            (f.Female_Only_Movies * 100/(COUNT(m.mid)*1.0)) AS Percentage_Female_Only_Movies
        FROM Movie m
        
            LEFT JOIN
                (
                    SELECT m.Year, COUNT(*) AS Female_Only_Movies FROM Movie m
                    
                        JOIN
                            (
                                SELECT DISTINCT MID FROM M_Cast 
                                WHERE MID NOT IN
                                (
                                    SELECT m_c.MID FROM M_Cast m_c
                                    JOIN Person p ON p.PID = TRIM(m_c.PID)
                                    WHERE p.Gender = 'Male'
                                )
                            )c ON c.mid = m.mid
                            GROUP BY m.year
                            
                )f ON m.year = f.year 
                GROUP BY m.year
               
        """

q5b = pd.read_sql_query(query, conn)    
print(q5b.shape)
q5b.head()

(125, 4)
CPU times: user 224 ms, sys: 8.23 ms, total: 232 ms
Wall time: 239 ms


,Release_Year,No_of_Movies,Female_Only_Movies,Percentage_Female_Only_Movies
0,1931,1,NaN,NaN
1,1936,3,NaN,NaN
2,1939,2,1.0,50.0
3,1941,1,NaN,NaN
4,1943,1,NaN,NaN


## Q6 --- Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [93]:
%%time

query = """
        SELECT m.Title AS Movie_Name, MAX(c.Cast) AS Cast_Size
        
        FROM(
                SELECT COUNT(*) AS Cast,MID
                FROM M_Cast GROUP BY MID
            )c
        
        JOIN Movie m on m.MID = c.MID  

        """

q6 = pd.read_sql_query(query, conn)    
print(q6.shape)
q6.head()

(1, 2)
CPU times: user 57.3 ms, sys: 9.45 ms, total: 66.7 ms
Wall time: 67 ms


,Movie_Name,Cast_Size
0,Ocean's Eight,238


## Q7 --- A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D.

In [124]:
%%time

query = """
        SELECT m.Year AS Begin, m.Year+9 AS Till, COUNT(*) No_of_Movies 
        FROM(
                SELECT DISTINCT Year from Movie
            )m
        
        JOIN Movie mo ON mo.Year >= Begin and mo.Year<= Till
        GROUP BY Till ORDER BY No_of_Movies DESC LIMIT 1
        
        """

q7 = pd.read_sql_query(query, conn)    
print(q7.shape)
q7.head()

(1, 3)
CPU times: user 104 ms, sys: 2.71 ms, total: 107 ms
Wall time: 107 ms


,Begin,Till,No_of_Movies
0,2008,2017,1128


## Q8 --- Find all the actors that made more movies with Yash Chopra than any other director.

In [174]:
%%time

query = """
        SELECT s.*, (s.Next_Release - s.Release_Year) AS Break
        
            FROM
                ( SELECT s1.Name AS Actor_Name, s1.year AS Release_Year,
                  LEAD(s1.year, 1, 0) OVER (PARTITION BY s1.name ORDER BY s1.year ASC) AS Next_Release
            
            FROM 
                (
                    ( SELECT DISTINCT TRIM(pid) AS pid, TRIM(Name) AS Name FROM Person ) p
                                        
                    JOIN
                        ( SELECT DISTINCT TRIM(mid) AS mid, trim(pid) AS pid FROM M_Cast )mc 
                            ON p.pid = mc.pid 
                    JOIN
                        ( SELECT TRIM(mid) AS mid, trim(title) AS title, TRIM(year) AS year FROM Movie)m 
                            ON m.mid = mc.mid
                )s1  
                )s 
                
            WHERE s.Next_Release > 0 AND (s.Next_Release - s.Release_Year) < 3   
        """

q8 = pd.read_sql_query(query, conn)    
print(q8.shape)
q8.head()

(42919, 4)
CPU times: user 926 ms, sys: 59.9 ms, total: 986 ms
Wall time: 1.01 s


,Actor_Name,Release_Year,Next_Release,Break
0,A. Abdul Hameed,1974,1975,1
1,A.K. Hangal,1966,1967,1
2,A.K. Hangal,1967,1969,2
3,A.K. Hangal,1969,1971,2
4,A.K. Hangal,1971,1971,0


## Q9 --- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [139]:
%%time

query = """
        SELECT Name AS Actor_Name FROM Person
        WHERE PID IN 
            ( SELECT TRIM(PID) FROM M_Cast 
                WHERE MID IN 
                ( Select MID FROM M_Cast 
                    WHERE TRIM(PID) IN 
                    ( Select TRIM(PID) AS PID FROM M_Cast 
                        WHERE MID IN 
                        ( SELECT m.MID FROM Movie m
                        
        INNER JOIN M_Cast c ON c.MID = m.MID
        INNER JOIN Person p ON TRIM(p.PID) = TRIM(c.PID)
        WHERE TRIM(Name) = 'Shah Rukh Khan'))))

        """

q9 = pd.read_sql_query(query, conn)    
print(q9.shape)
q9.head()

(28081, 1)
CPU times: user 361 ms, sys: 17 ms, total: 378 ms
Wall time: 378 ms


,Actor_Name
0,Freida Pinto
1,Rohan Chand
2,Damian Young
3,Waris Ahluwalia
4,Caroline Christl Long


In [175]:
# Reference: https://github.com/shshankar1